# 텍스트 분류 실습 - 20 뉴스그룹 분류

사이킷런이 내부에 가지고 있는 예제 데이터인 20 뉴스그룹 데이터 세트 이용

텍스트 분류 : 특정 문서의 분류를 학습 데이터를 통해 학습 -> 모델 생성 -> 학습 모델을 이용해 다른 문서의 분류 예측

사이킷런은 fetch_20newsgroups() API를 이용해 뉴스 그룹의 분류를 수행해 볼 수 있는 예제 데이터 제공

텍스트를 피처 벡터화로 변환하면 일반적으로 희소 행렬 형태가 됨

희소 행렬에 분류를 효과적으로 잘 처리할 수 있는 알고리즘 : 로지스틱 회귀, 선형 SVM, 나이브 베이즈 등


텍스트 기반 분류 : 텍스트 정규화 -> 피처 벡터화 -> 적합한 머신러닝 알고리즘 적용 -> 분류를 학습/예측/평가

### 텍스트 정규화
fetch_20newsgroups()를 인터넷에서 로컬 컴퓨터로 데이터 내려받음 -> 메모리로 데이터 로딩

In [1]:
from sklearn.datasets import fetch_20newsgroups

news_data = fetch_20newsgroups(subset='all',random_state=156)

fetch_20newsgroups()는 파이썬 딕셔너리와 유사한 Bunch 객체를 반환함

어떠한 key 값을 가지고 있는지 확인

In [3]:
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


load_xxx() API와 유사한 Key 값을 가지고 있음

filenames : API가 인터넷에서 내려 받아 로컬 컴퓨터에 저장하는 디렉터리와 파일명
    
Target 클래스가 어떻게 구성되어 있는지 확인

In [5]:
import pandas as pd

print('target 클래스의 값과 분포도 \n',pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들 \n',news_data.target_names)

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


Target 클래스 : 0~19, 20개로 구성됨

개별 데이터가 텍스트로 어떻게 구성되어 있는지 데이터를 한 개만 추출하여 값 확인

In [6]:
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

뉴스그룹 기사 내용, 뉴스그룹 제목, 작성자, 소속, 이메일 등의 다양한 정보를 가짐

내용을 제외하고 제목 등의 다른 정보는 제거함 (제목, 소속, 이메일 등 정보들은 뉴스그룹 분류의 Target 클래스 값과 유사한 데이터를 가지고 있는 경우가 많기 때문)

위 피처들을 포함하면 상당히 높은 예측 성능을 나타냄 -> 텍스트 분석 의도를 벗어나기에 순수한 텍스트로만 구성된 기사 내용으로 뉴스 그룹 분류 수행함

remove 파라미터를 사용하여 뉴스 그룹 기사의 header, footer 등을 제거함

fetch_20newsgroups()는 subset 파라미터를 이용하여 학습 데이터 세트, 테스트 데이터 세트를 분리해 내려받을 수 있음

In [7]:
from sklearn.datasets import fetch_20newsgroups

# subset='train'으로 학습용(Train) 데이터만 추출, remove=('headers', 'footers', 'quotes')로 내용만 추출
train_news= fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), random_state=156)
X_train = train_news.data
y_train = train_news.target
print(type(X_train))

# subset='test'으로 테스트(Test) 데이터만 추출, remove=('headers', 'footers', 'quotes')로 내용만 추출
test_news= fetch_20newsgroups(subset='test',remove=('headers', 'footers','quotes'),random_state=156)
X_test = test_news.data
y_test = test_news.target
print('학습 데이터 크기 {0} , 테스트 데이터 크기 {1}'.format(len(train_news.data) , len(test_news.data)))

<class 'list'>
학습 데이터 크기 11314 , 테스트 데이터 크기 7532


### 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가

학습 데이터 : 11314개의 뉴스그룹 문서 (리스트 형태)
테스트 데이터 : 7532개의 문서 (리스트 형태)

CountVectorizer를 이용하여 학습 데이터의 텍스트를 피처 벡터화함

테스트 데이터 피처 벡터화 시 유의할 점 : 반드시 학습 데이터를 이용해 fit()이 수행된 CountVectorizer 객체를 이용하여 테스트 데이터를 변환해야 함
=> 이렇게 해야 학습 시 설정된 CountVectorizer의 피처 개수와 테스트 데이터를 CountVectorizer로 변환할 피처 개수가 같아짐

테스트 데이터 피처 벡터화 : 학습 데이터에 사용된 CountVectorizer 객체 변수인 cnt_vect.transform()을 이용해 변환

테스트 데이터 피처 벡터화 시 fit_transform()을 사용하면 안 됨 (학습 시 사용된 피처 개수와 예측 시 사용할 피처 개수가 달라지기 때문)

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorization으로 feature extraction 변환 수행. 
cnt_vect = CountVectorizer()

cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

# 학습 데이터로 fit( )된 CountVectorizer를 이용하여 테스트 데이터를 feature extraction 변환 수행. 
X_test_cnt_vect = cnt_vect.transform(X_test)

print('학습 데이터 Text의 CountVectorizer Shape:',X_train_cnt_vect.shape)

학습 데이터 Text의 CountVectorizer Shape: (11314, 101631)


피처 추출 결과 11314개의 문서에서 단어가 101631개로 만들어짐

피처 벡터화된 데이터에 로지스틱 회귀를 적용하여 뉴스그룹에 대한 분류 예측 수행

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

# LogisticRegression을 이용하여 학습/예측/평가 수행. 
lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train_cnt_vect , y_train)
pred = lr_clf.predict(X_test_cnt_vect)
print('CountVectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

CountVectorized Logistic Regression 의 예측 정확도는 0.617


Count 기반으로 피처 벡터화가 적용된 데이터 세트에 대한 로지스틱 회귀의 예측 정확도 : 약 0.616
    
Count 기반에서 TF-IDF 기반으로 벡터화 변경하여 예측 모델 수행

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF Vectorization 적용하여 학습 데이터셋과 테스트 데이터 셋 변환. 
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

# LogisticRegression을 이용하여 학습/예측/평가 수행. 
lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train_tfidf_vect , y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))


TF-IDF Logistic Regression 의 예측 정확도는 0.678


정확도 : TF-IDF > 단순 카운트 기반

일반적으로 문서 내 텍스트가 많고, 많은 문서를 가지는 텍스트 분석에서 TF-IDF 벡터화가 더 좋은 예측 결과 도출

텍스트 분석에서 머신러닝 모델의 성능을 향상시키는 중요한 2가지 방법

1. 최적의 ML 알고리즘을 선택하는 것
2. 최상의 피처 전처리를 수행하는 것

앞의 TF-IDF 벡터화의 파라미터를 변경함

TfidfVectorizer 클래스의 스톱 워드를 'english'로, ngram_range(1,2)로, max_df=300으로 변경한 뒤 예측 성능 재측정

In [12]:
# stop words 필터링을 추가하고 ngram을 기본(1,1)에서 (1,2)로 변경하여 Feature Vectorization 적용.
tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300 )
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train_tfidf_vect , y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

TF-IDF Vectorized Logistic Regression 의 예측 정확도는 0.690


GridSearchCV를 이용하여 로지스틱 회귀의 하이퍼 파라미터 최적화 수행

로지스틱 회귀의 C 파라미터만 변경하여 최적의 C 값 찾음 -> C값으로 학습된 모델에서 테스트 데이터로 예측해 성능 평가

In [13]:
from sklearn.model_selection import GridSearchCV

# 최적 C 값 도출 튜닝 수행. CV는 3 Fold셋으로 설정. 
params = { 'C':[0.01, 0.1, 1, 5, 10]}
grid_cv_lr = GridSearchCV(lr_clf ,param_grid=params , cv=3 , scoring='accuracy' , verbose=1 )
grid_cv_lr.fit(X_train_tfidf_vect , y_train)
print('Logistic Regression best C parameter :',grid_cv_lr.best_params_ )

# 최적 C 값으로 학습된 grid_cv로 예측 수행하고 정확도 평가. 
pred = grid_cv_lr.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

Fitting 3 folds for each of 5 candidates, totalling 15 fits
Logistic Regression best C parameter : {'C': 10}
TF-IDF Vectorized Logistic Regression 의 예측 정확도는 0.704


로지스틱 회귀의 C가 10일 때 가장 좋은 예측 성능을 가짐
이 C값을 테스트 데이터 세트에 적용하여 0.703으로 이전보다 약간 향상된 성능 수치를 가짐

### 사이킷런 파이프라인 사용 및 GridSearchCV와의 결합
사이킷런의 Pipeline 클래스를 이용하여 피처 벡터화, ML 알고리즘 학습/예측을 위한 코드 작성을 한 번에 진행 가능

파이프라인을 이용하면 데이터 전처리, 머신러닝 학습 과정을 통일된 API 기반에서 처리할 수 있음 -> 더 직관적인 ML 모델 코드 생성 가능

대용량 데이터의 피처 벡터화 결과를 별도 데이터로 저장하지 않고 스트림 기반에서 바로 머신러닝 알고리즘의 데이터로 입력할 수 있음 -> 수행시간 절약

일반적으로 사이킷런 파이프라인 : 텍스트 기반 피처 벡터화 + 모든 데이터 전처리 작업과 Estimator 결합

예) 스케일링, 벡터 정규화, PCA 등의 변환 작업 + 분류, 회귀 등의 Estimator 한 번에 결합

위의 코드를 pipeline을 이용해 다시 작성

In [15]:
from sklearn.pipeline import Pipeline

# TfidfVectorizer 객체를 tfidf_vect 객체명으로, LogisticRegression객체를 lr_clf 객체명으로 생성하는 Pipeline생성
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)),
    ('lr_clf', LogisticRegression(solver='liblinear', C=10))
])

# 별도의 TfidfVectorizer객체의 fit_transform( )과 LogisticRegression의 fit(), predict( )가 필요 없음. 
# pipeline의 fit( ) 과 predict( ) 만으로 한꺼번에 Feature Vectorization과 ML 학습/예측이 가능. 
pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)
print('Pipeline을 통한 Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

Pipeline을 통한 Logistic Regression 의 예측 정확도는 0.704


TfidfVectorizer 객체를 tfidf_vect라는 객체 변수명으로, LogisticRegression 객체를 lr_clf라는 객체 변수명으로 생성 -> 두 객체를 파이프라인으로 연결하는 Pipeline 객체 pipeline을 생성한다는 의미

Pipeline의 fit()과 predict()로 통일되어 수행됨

사이킷런은 Pipeline 기반에서도 하이퍼 파라미터 튜닝을 GridSearchCV 방식으로 진행할 수 있도록 지원함

피처 벡터화를 위한 파라미터와 ML 알고리즘을 위한 하이퍼 파라미터를 모두 한 번에 GridSearchCV를 이용해 최적화할 수 있음

GridsearchCV에 Pipeline을 입력하면서 TfidfVectorizer의 파라미터와 Logistic Regression의 하이퍼 파라미터를 함께 최적화함

Pipeline+GridsearchCV를 적용할 때 유의할 점 : 모두의 파라미터를 최적화하기에 너무 많은 튜닝 시간이 필요함

따라서, 다음 예제에서는 27개의 파라미터 경우의 수 * 3개의 CV로 총 81번의 학습과 검증을 수행함

In [16]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english')),
    ('lr_clf', LogisticRegression(solver='liblinear'))
])

# Pipeline에 기술된 각각의 객체 변수에 언더바(_)2개를 연달아 붙여 GridSearchCV에 사용될 
# 파라미터/하이퍼 파라미터 이름과 값을 설정. . 
params = { 'tfidf_vect__ngram_range': [(1,1), (1,2), (1,3)],
           'tfidf_vect__max_df': [100, 300, 700],
           'lr_clf__C': [1, 5, 10]
}

# GridSearchCV의 생성자에 Estimator가 아닌 Pipeline 객체 입력
grid_cv_pipe = GridSearchCV(pipeline, param_grid=params, cv=3 , scoring='accuracy',verbose=1)
grid_cv_pipe.fit(X_train , y_train)
print(grid_cv_pipe.best_params_ , grid_cv_pipe.best_score_)

pred = grid_cv_pipe.predict(X_test)
print('Pipeline을 통한 Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))


Fitting 3 folds for each of 27 candidates, totalling 81 fits
{'lr_clf__C': 10, 'tfidf_vect__max_df': 700, 'tfidf_vect__ngram_range': (1, 2)} 0.7550828826229531
Pipeline을 통한 Logistic Regression 의 예측 정확도는 0.702


TfidfVectoorizer 객체의 max_df=700, ngram_range(1,2)로 피처 벡터화된 데이터 세트에 LogisticRegression의 C 하이퍼 파라미터에 10을 적용해 예측 분류를 수행할 때 가장 좋은 검증 세트 성능 수치 도출됨

하지만 최적화한 파라미터를 기반으로 테스트 데이터 세트에 대한 예측을 했을 때 정확도는 0.702로 크게 개선되지 않음